# Visualization

In [25]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import transform

## Visualizing Image and Steering angle

In [131]:
def show_sample(sample,pred_angle):
    r""" Helper function for (batch) sample visualization
    随机显示一个batch里面的五张图
    Args:
        sample: Dictionary
    """
    image_dims = len(sample['image'].shape)
    assert image_dims <= 5, "Unsupported image shape: {}".format(sample['image'].shape)
    if image_dims == 3:
        fig, ax = plt.subplots()
        ax.imshow(sample['image'].permute(1,2,0))
    if image_dims == 4:
        n = sample['image'].shape[0]
        sample['image'] = torch.Tensor(resize(sample['image'], (n,3,480,640),anti_aliasing=True))
        images = sample['image'].permute(0,2,3,1)
        fig = plt.figure(figsize=(55, 35))
        for i in range(n):
            ax = fig.add_subplot(10,5,i+1)
            ax.imshow(images[i])
            ax.axis('off')
            ax.set_title("t={}".format(sample['timestamp'][i]))
            ax.text(10, 30, sample['frame_id'][i], color='red')
            ax.text(10, 430, "man-angle {:.3}".format(sample['angle'][i].item()), color='red')
            ax.text(10, 470, "pred-angle {:.3}".format(pred_angle[i].item()), color='red')
    else:
        sample['image'] = sample['image'].permute(0,2,1,3,4)
        batch_size,seq_len,channel = sample['image'].shape[0],sample['image'].shape[1],sample['image'].shape[2]
        sample['image'] = torch.Tensor(resize(sample['image'], (batch_size,seq_len,3,480,640),anti_aliasing=True))
        n0 = sample['image'].shape[0]
        n1 = sample['image'].shape[1] if image_dims == 5 else 1
        images_flattened = torch.flatten(sample['image'], end_dim=-4)
        fig, axis = plt.subplots(n0, n1, figsize=(25, 15))
        for i1 in range(n1):
            for i0 in range(n0):
                image = images_flattened[i0 * n1 + i1]
                axis = ax[i0, i1]
                axis.imshow(image.permute(1,2,0))
                axis.axis('off')
                axis.set_title("t={}".format(sample['timestamp'][i0][i1]))
                axis.text(10, 30, sample['frame_id'][i0][i1], color='red')
                axis.text(10, 430, "man-angle {:.3}".format(sample['angle'][i0][i1].item()), color='red')
                axis.text(10, 470, "pred-angle {:.3}".format(pred_angle[i0][i1].item()), color='red')
    

In [136]:
rsrc = \
 [[43.45456230828867, 118.00743250075844],
  [104.5055617352614, 69.46865203761757],
  [114.86050156739812, 60.83953551083698],
  [129.74572757609468, 50.48459567870026],
  [132.98164627363735, 46.38576532847949],
  [301.0336906326895, 98.16046448916306],
  [238.25686790036065, 62.56535881619311],
  [227.2547443287154, 56.30924933427718],
  [209.13359962247614, 46.817221154818526],
  [203.9561297064078, 43.5813024572758]]
rdst = \
 [[10.822125594094452, 1.42189132706374],
  [21.177065426231174, 1.5297552836484982],
  [25.275895776451954, 1.42189132706374],
  [36.062291434927694, 1.6376192402332563],
  [40.376849698318004, 1.42189132706374],
  [11.900765159942026, -2.1376192402332563],
  [22.25570499207874, -2.1376192402332563],
  [26.785991168638553, -2.029755283648498],
  [37.033067044190524, -2.029755283648498],
  [41.67121717733509, -2.029755283648498]]

tform3_img = transform.ProjectiveTransform()
tform3_img.estimate(np.array(rdst), np.array(rsrc))

def calc_curvature(v_ego, angle_steers, angle_offset=0):
    deg_to_rad = np.pi/180.
    slip_fator = 0.0014 # slip factor obtained from real data
    steer_ratio = 15.3  # from http://www.edmunds.com/acura/ilx/2016/road-test-specs/
    wheel_base = 2.67   # from http://www.edmunds.com/acura/ilx/2016/sedan/features-specs/

    angle_steers_rad = (angle_steers - angle_offset) * deg_to_rad
    curvature = angle_steers_rad/(steer_ratio * wheel_base * (1. + slip_fator * v_ego**2))
    return curvature

def calc_lookahead_offset(v_ego, angle_steers, d_lookahead, angle_offset=0):
    #*** this function returns the lateral offset given the steering angle, speed and the lookahead distance
    curvature = calc_curvature(v_ego, angle_steers, angle_offset)

    # clip is to avoid arcsin NaNs due to too sharp turns
    y_actual = d_lookahead * np.tan(np.arcsin(np.clip(d_lookahead * curvature, -0.999, 0.999))/2.)
    return y_actual, curvature

def show_steering_angle_single(ax, angle, image_width, image_height, speed_ms, color=None, alpha=0.5):
    path_x = np.arange(0., 50.1, 0.5)
    path_y, _ = calc_lookahead_offset(speed_ms, angle, path_x)
    
    plot_points_x = []
    plot_points_y = []
    
    for x, y in zip(path_x, path_y):
        col, row = tform3_img((x, y))[0] # transform from car to image coordinate
        row = row / 160 * image_height
        col = col / 320 * image_width
        if row >= 0 and row <= image_height and col >= 0 and col <= image_width:
            plot_points_x.append(col)
            plot_points_y.append(row)
    
    ax.scatter(plot_points_x, plot_points_y, alpha=alpha, color=color)

In [138]:
def show_image_single(ax, image, 
                      angle_pred=None, angle_true=None, speed=None, alpha_angle=0.5,
                      cam_image=None, alpha_cam=0.4,
                      timestamp=None):
    assert len(image.shape) == 3, "Image should be 3-dimentional"
    ax.imshow(image)
    image_height, image_width, _ = image.shape
    line_space = 10 * image_height / 120
    if cam_image is not None:
        ax.imshow(cam_image, alpha=alpha_cam)
    if angle_pred is not None and speed is not None:
        show_steering_angle_single(ax, angle_pred, image_width, image_height, speed, color='red', alpha=alpha_angle)
        ax.text(10, image_height-line_space, "pred-angle: {:.4}".format(angle_pred), color='red')
    if angle_true is not None and speed is not None:
        show_steering_angle_single(ax, angle_true, image_width, image_height, speed, color='blue', alpha=alpha_angle)
        ax.text(10, image_height-2*line_space, "true-angle: {:.4}".format(angle_true), color='blue')
    if angle_pred is not None and angle_true is not None:
        ax.text(10, image_height-3*line_space, "error: {:.4}".format(angle_pred-angle_true), color='green')
    if timestamp is not None:
        ax.text(10, 10, "t={}".format(timestamp))

## Visualizing CNN filters

In [1]:
def visualize_cnn(cnn, max_row=10, max_col=10, select_channel=None, tb_writer=None, save_path=None):
    assert isinstance(cnn, nn.Conv3d) or isinstance(cnn, nn.Conv2d)
    
    filters = cnn.weight.cpu().detach().numpy() # output_ch x input_ch x D x H x W
    output_ch = np.minimum(cnn.weight.shape[0], max_col)
    input_ch = np.minimum(cnn.weight.shape[1], max_row)
    print(filters.shape)
    
    if isinstance(cnn, nn.Conv3d):
        if select_channel:
            assert isinstance(select_channel, int)
            filters = filters[:, :, select_channel, :, :]
        else:
            filters = np.mean(filters[:, :, :, :, :], axis=2)
    
    plt_idx = 0
    fig = plt.figure(figsize=(output_ch, input_ch))
    plt.xlabel("Output Channel")
    plt.ylabel("Input Channel")
    frame1 = plt.gca()
    frame1.axes.xaxis.set_ticklabels([])
    frame1.axes.yaxis.set_ticklabels([])
    for o in range(output_ch):
        for i in range(input_ch):
            image = filters[o, i, :, :]
            plt_idx += 1
            ax = fig.add_subplot(input_ch, output_ch, plt_idx)
            ax.imshow(image)
            ax.axis('off')
#     plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()


## GradCAM (class activation mapping)

In [58]:
from skimage.transform import resize

class CamExtractor():
    """
        Extracts cam features from the model
    """
    def __init__(self, model, register_hooks=True):
        self.model = model
        self.grad = None
        self.conv_output = None
        
        if register_hooks:
            self.register_hooks()
        
    def gradient_hook(self, model, grad_input, grad_output):
        self.grad = grad_output[0].cpu().detach().numpy()
        
    def conv_output_hook(self, model, input, output):
        self.conv_output = output.cpu().detach().numpy()
        
    def register_hooks(self):
        raise NotImplementedError("You should implement this method for your own model!")
        
    def forward(self, x):
        raise NotImplementedError("You should implement this method for your own model!")
        
    def to_image(self, height=None, width=None):
        assert self.grad is not None and self.conv_output is not None, "You should perform both forward pass and backward propagation first!"
        # both grad and conv_output should have the same dimension of: (*, channel, H, W)
        # we produce image(s) of shape: (*, H, W)
        channel_weight = np.mean(self.grad, axis=(-2, -1)) # *, channel
        conv_permuted = np.moveaxis(self.conv_output, [-2, -1], [0, 1]) # H, W, *, channel
        cam_image_permuted = channel_weight * conv_permuted # H, W, *, channel
        cam_image_permuted = np.mean(cam_image_permuted, axis=-1) # H, W, *
        cam_image = np.moveaxis(cam_image_permuted, [0, 1], [-2, -1]) # *, H, W
        
        if height is not None and width is not None:
            image_shape = list(cam_image.shape)
            image_shape[-2] = height
            image_shape[-1] = width
            cam_image = resize(cam_image, image_shape)
        return cam_image
        
    
class CamExtractorTLModel(CamExtractor):
    
    def register_hooks(self):
        self.model.ResNet.layer4.register_forward_hook(self.conv_output_hook)
        self.model.ResNet.layer4.register_backward_hook(self.gradient_hook)

class CamExtractor3DCNN(CamExtractor):
    
    def gradient_hook(self, model, grad_input, grad_output):
        grad = grad_output[0].cpu().detach().numpy()
        self.grad = np.moveaxis(grad, 1, 2) # restore old dimension (batch x seq_len x channel x H x W)
        
    def conv_output_hook(self, model, input, output):
        conv_output = output.cpu().detach().numpy()
        self.conv_output = np.moveaxis(conv_output, 1, 2) # restore old dimension (batch x seq_len x channel x H x W)
    
    def register_hooks(self):
        self.model.Convolution6.register_forward_hook(self.conv_output_hook)
        self.model.Convolution6.register_backward_hook(self.gradient_hook)
    

In [18]:
def visualize_gradcam(cam_image, sample_image, save_image=None):
    """
    Show CAM image as an overlay to sample image
    
    Args:
        cam_image: image extracted from CamExtractor.to_image(), shape: (*, H, W)
        sample_image: the image fed into the model, shape: (batch_size, seq_len, channel, H, W) or (batch_size, channel, H, W) if seq_len == 1
    """
    
    n_dims = len(cam_image.shape)
    assert n_dims == len(sample_image.shape) - 1 and (n_dims == 4 or n_dims == 3), "cam_image dim: {} but sample_image dim: {}".format(cam_image.shape, sample_image.shape)
    
    batch_size = cam_image.shape[0]
    assert batch_size == sample_image.shape[0], "batch size of sample image and cam image must agree!"
    
    if n_dims == 4:
        seq_len = cam_image.shape[1]
        assert seq_len == sample_image.shape[1], "seq_len of sample image and cam image must agree!"
    else:
        seq_len = 1
    
    fig = plt.figure(figsize=(seq_len*6, batch_size*4))
    plt.xlabel("seq_idx")
    plt.ylabel("batch_idx")
    frame1 = plt.gca()
    frame1.axes.xaxis.set_ticklabels([])
    frame1.axes.yaxis.set_ticklabels([])
    
    plt_idx = 1
    for b in range(batch_size):
        for s in range(seq_len):
            cam_image_cur = cam_image[b, s, :, :] if n_dims == 4 else cam_image[b, :, :]
            sample_image_cur = sample_image[b, s, :, :, :] if n_dims == 4 else sample_image[b, :, :, :]
            sample_image_cur = sample_image_cur.permute(1, 2, 0) # putting color dim in the end, as required by matplotlib
            
            ax = fig.add_subplot(batch_size, seq_len, plt_idx)
            ax.imshow(sample_image_cur)
            ax.imshow(cam_image_cur, cmap='jet', alpha=0.4)
            ax.axis('off')
            
            plt_idx += 1
            
    plt.tight_layout()
    if save_image:
        plt.savefig(save_image, format='png')
        plt.close()
    else:
        plt.show()